In [ ]:
# Importações
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# 1. Carregar os datasets
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
gender = pd.read_csv("gender_submission.csv")   # contém Survived real do test.csv

print("\nPrimeiras linhas do train.csv:")
print(train.head())

print("\nInformações do dataset de treino:")
print(train.info())

# 2. Tratamento simples de dados
# Preencher valores ausentes
train["Age"] = train["Age"].fillna(train["Age"].median())
test["Age"] = test["Age"].fillna(test["Age"].median())

train["Fare"] = train["Fare"].fillna(train["Fare"].median())
test["Fare"] = test["Fare"].fillna(test["Fare"].median())

train["Embarked"] = train["Embarked"].fillna(train["Embarked"].mode()[0])
test["Embarked"] = test["Embarked"].fillna(test["Embarked"].mode()[0])

# Excluir colunas não úteis (texto, código, etc.)
colunas_remover = ["Name", "Ticket", "Cabin"]
train = train.drop(columns=colunas_remover)
test = test.drop(columns=colunas_remover)

# Transformar Sex para número
train["Sex"] = train["Sex"].map({"male": 0, "female": 1})
test["Sex"] = test["Sex"].map({"male": 0, "female": 1})

# One-hot para Embarked
train = pd.get_dummies(train, columns=["Embarked"])
test = pd.get_dummies(test, columns=["Embarked"])

# 3. Definir X e y
X = train.drop("Survived", axis=1)
y = train["Survived"]

X_test_real = test.copy()  # será usado para prever

# 4. Normalização simples
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test_real)

# 5. Dividir em treino e validação (80/20)
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, test_size=0.20, stratify=y, random_state=42
)

# 6. Modelos simples
modelos = {
    "Regressão Logística": LogisticRegression(max_iter=1000),
    "Árvore de Decisão": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=200),
    "Gradient Boosting": GradientBoostingClassifier()
}

resultados = {}

# 7. Treinar e avaliar cada modelo
for nome, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_val)

    acc = accuracy_score(y_val, y_pred)
    prec = precision_score(y_val, y_pred)
    rec = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    resultados[nome] = {
        "Acurácia": acc,
        "Precisão": prec,
        "Recall": rec,
        "F1-Score": f1
    }

# Mostrar métricas
print("\n==== RESULTADOS NA VALIDAÇÃO ====")
for nome, met in resultados.items():
    print(f"\nModelo: {nome}")
    for m, v in met.items():
        print(f"  {m}: {v:.4f}")

# 8. Escolher melhor modelo baseado em F1
melhor = max(resultados.items(), key=lambda x: x[1]["F1-Score"])
melhor_modelo_nome = melhor[0]
print("\nMelhor modelo segundo F1-Score:", melhor_modelo_nome)

# 9. Previsão no dataset de teste
melhor_modelo = modelos[melhor_modelo_nome]
y_pred_test = melhor_modelo.predict(X_test_scaled)

# Comparar com valores reais do gender_submission
y_real_test = gender["Survived"]

acc_test = accuracy_score(y_real_test, y_pred_test)
prec_test = precision_score(y_real_test, y_pred_test)
rec_test = recall_score(y_real_test, y_pred_test)
f1_test = f1_score(y_real_test, y_pred_test)

print("\n==== DESEMPENHO FINAL (Dataset Teste) ====")
print("Acurácia:", acc_test)
print("Precisão:", prec_test)
print("Recall:", rec_test)
print("F1-Score:", f1_test)

{'RegressaoLogistica': {'acuracia': 0.9186602870813397,
  'precisao': 0.8511904761904762,
  'recall': 0.9407894736842105,
  'f1': 0.89375,
  'roc_auc': np.float64(0.9603037198258805),
  'confusao': array([[241,  25],
         [  9, 143]])},
 'FlorestaAleatoria': {'acuracia': 0.80622009569378,
  'precisao': 0.7177914110429447,
  'recall': 0.7697368421052632,
  'f1': 0.7428571428571429,
  'roc_auc': np.float64(0.9019464780371983),
  'confusao': array([[220,  46],
         [ 35, 117]])},
 'GradientBoosting': {'acuracia': 0.854066985645933,
  'precisao': 0.7898089171974523,
  'recall': 0.8157894736842105,
  'f1': 0.8025889967637541,
  'roc_auc': np.float64(0.9200880490700435),
  'confusao': array([[233,  33],
         [ 28, 124]])},
 'KNN': {'acuracia': 0.854066985645933,
  'precisao': 0.7757575757575758,
  'recall': 0.8421052631578947,
  'f1': 0.807570977917981,
  'roc_auc': np.float64(0.921633854372774),
  'confusao': array([[229,  37],
         [ 24, 128]])}}